In [1]:
import logging
import sys
import os
sys.path.append(r"C:\Users\pasqu\Desktop\progettoasnm")

from Code.notebook.graph.GraphConstructor import GraphConstructor
from preprocessing.text_preprocessor import TextPreprocessor
from models.clustering import Clustering
from visualization.cluster_visualizer import ClusterVisualizer
from models.sentiment_analysis import SentimentAnalyzer
from visualization.sentiment_visualizer import SentimentVisualizer
from visualization.wordcloud_visualizer import WordCloudVisualizer
from visualization.lda_visualizer import LDAViz
from models.topic_modeling import TopicModeling


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\pasqu\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\pasqu\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
c:\Users\pasqu\AppData\Local\Programs\Python\Python312\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
logging.info("Inizio del processo principale.")
output_dir = r"Code//notebook//Analisi della polarizzazione dei contenuti//output"
# Costruzione del grafo
graph_builder = GraphConstructor(
        followers_paths= ["../../../Code/data_extraction/followers.csv"],
        data_paths = ["../../../Code/data_extraction/total_post1TS.csv"],
        info_filepath="graph_info.json",
        centralities_filepath="centralities_info.json",
    )
graph_builder.build_graph()
graph = graph_builder.graph

# # Estrazione e pre-elaborazione dei testi
preprocessor = TextPreprocessor()
user_opinions = preprocessor.extract_user_opinions(graph_builder)

In [5]:
sentiment_scores = SentimentAnalyzer().extract_sentiments_from_graph(graph_builder)

c:\Users\pasqu\Desktop\progettoasnm\Code\notebook\Analisi della polarizzazione dei contenuti\models\sentiment_analysis.py:41: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  sentiment_scores = df_data.groupby("thread_user_pk").apply(


In [6]:
sentiment_scores

{'10016217875.0': 0.5,
 '1006068859.0': 0.5,
 '10235026849.0': 0.5,
 '10265535245.0': 0.5,
 '10329650804.0': 0.5,
 '10356454179.0': 0.5,
 '10357387646.0': 0.5,
 '10439803209.0': 0.5,
 '104793531.0': 0.5,
 '1055174331.0': 0.5,
 '10632785952.0': 0.5,
 '1069734614.0': 0.5,
 '10831594916.0': 0.5,
 '10911266568.0': 0.5,
 '1093971766.0': 0.5,
 '1095525944.0': 0.5,
 '10968710940.0': 0.5,
 '11043121699.0': 0.5,
 '11093858195.0': 0.5,
 '111380.0': 0.5,
 '11166093367.0': 0.5,
 '11200430.0': 0.5,
 '11344047012.0': 0.5,
 '11397047350.0': 0.5,
 '1141192858.0': 0.5,
 '11426157437.0': 0.5,
 '11456243.0': 0.5,
 '11478254.0': 0.5,
 '1154211412.0': 0.5,
 '11555809545.0': 0.5,
 '11641914726.0': 0.5,
 '11672361316.0': 0.5,
 '11693141898.0': 0.5,
 '11702818748.0': 0.5,
 '1189366729.0': 0.5,
 '1194114589.0': 0.5,
 '11945267122.0': 0.5,
 '11953797989.0': 0.5,
 '1200616021.0': 0.5,
 '1202687360.0': 0.5,
 '1205482163.0': 0.5,
 '12061848388.0': 0.5,
 '1209032666.0': 0.5,
 '12111591662.0': 0.5,
 '12117726138.0':

In [7]:
# Clustering
clustering = Clustering()
cluster_labels = clustering.cluster(user_opinions, method="dbscan")


MemoryError: Unable to allocate 25.0 GiB for an array with shape (15989, 209723) and data type float64

In [6]:
# Clustering
clustering = Clustering()
cluster_labels = clustering.cluster(user_opinions, method="dbscan")

# Visualizzazione dei cluster
cluster_visualizer = ClusterVisualizer(output_dir=output_dir)
cluster_visualizer.visualize(user_opinions, cluster_labels)

# Visualizzazione del sentiment
sentiment_visualizer = SentimentVisualizer(output_dir=output_dir)

# Visualizza la distribuzione del sentiment
sentiment_visualizer.visualize_sentiment_distribution(
sentiment_scores, cluster_labels
)

# Visualizza la mappa di calore sentiment vs temi
sentiment_visualizer.visualize_sentiment_vs_themes_heatmap(
sentiment_scores, user_opinions, cluster_labels
)

# Identificazione e visualizzazione temi polarizzanti

# UnGram
polarizing_words = clustering.identify_polarizing_themes(
user_opinions, cluster_labels
)
wordcloud_visualizer = WordCloudVisualizer()
wordcloud_visualizer.visualize(polarizing_words, output_dir, "Un")

# BiGram
polarizing_words = clustering.identify_polarizing_themes_bigram(
user_opinions, cluster_labels
)
wordcloud_visualizer.visualize(polarizing_words, output_dir, "Bi")

# Topic Modeling

topic_modeling = TopicModeling()
lda_model, dictionary, corpus = topic_modeling.perform_topic_modeling(
user_opinions, len(set(cluster_labels.values()))
)

# Visualizzazione dei temi
lda_visualizer = LDAViz()
lda_visualizer.visualize(lda_model, corpus, dictionary, output_dir=output_dir)

logging.info("Processo principale completato.")

MemoryError: Unable to allocate 5.45 GiB for an array with shape (7915, 92493) and data type float64

In [12]:
len(set(cluster_labels.values()))

24

In [13]:
polarizing_words = clustering.identify_polarizing_themes(
        user_opinions, cluster_labels
)

In [11]:
bigram = clustering.identify_polarizing_themes_bigram(
        user_opinions, cluster_labels
)


In [12]:
import re
for i in range(len(bigram)):
    bigram[i] = re.sub(" ","_", bigram[i])
    
bigram

['artificial_intelligence',
 'even_artificial',
 'manufacture_intelligence',
 'intelligence_collected',
 'manufactures_artificial',
 'humans_manufactures',
 'intelligence_humans',
 'collected_human',
 'cannot_manufacture',
 'good_morning',
 'art_threads',
 'machine_learning',
 'good_night',
 'learning_algorithms',
 'front_end',
 '𝑦𝑜𝑢_𝑏𝑎𝑐𝑘',
 '𝑌𝑜𝑢_𝑓𝑜𝑙𝑙𝑜𝑤',
 '𝑚𝑒_𝑓𝑜𝑙𝑙𝑜𝑤',
 '𝑓𝑜𝑙𝑙𝑜𝑤_𝑦𝑜𝑢',
 '𝑏𝑎𝑐𝑘_𝑌𝑜𝑢',
 '𝑏𝑎𝑐𝑘_ig',
 '𝑏𝑎𝑐𝑘_100',
 'neovim_𝑌𝑜𝑢',
 'code_neovim',
 'thank_you',
 'thank_service',
 'right_thank',
 'thank_god',
 'thank_talking',
 'talking_day',
 'thank_mba',
 'thank_mrs',
 'mba_illa',
 'nice_work',
 'nice_proposals',
 'nice_editing',
 'nice_color',
 'wow_nice',
 'tiktok_nice',
 'wow_tiktok',
 'nice_place',
 'amazing_beaches',
 'im_interested',
 'people_interested',
 'data_visualization',
 'hey_thread',
 'thread_day',
 'interested_add',
 'im_artist',
 'computing_python',
 'learning_python',
 'first_thread',
 'hello_world',
 'hello_friends',
 'hello_online',
 'hello_everyone',
 'friends

In [ ]:
wordcloud_visualizer = WordCloudVisualizer()
wordcloud_visualizer.visualize(polarizing_words, output_dir, "Bi")

In [1]:


def main():


    # Identificazione e visualizzazione temi polarizzanti

    # UnGram
    polarizing_words = clustering.identify_polarizing_themes(
        user_opinions, cluster_labels
    )
    wordcloud_visualizer = WordCloudVisualizer()
    wordcloud_visualizer.visualize(polarizing_words, output_dir, "Un")

    # BiGram
    polarizing_words = clustering.identify_polarizing_themes_bigram(
        user_opinions, cluster_labels
    )
    wordcloud_visualizer.visualize(polarizing_words, output_dir, "Bi")

    # TriGram
    polarizing_words = clustering.identify_polarizing_themes_trigram(
        user_opinions, cluster_labels
    )
    wordcloud_visualizer.visualize(polarizing_words, output_dir, "Tri")

    # Topic Modeling

    topic_modeling = TopicModeling()
    lda_model, dictionary, corpus = topic_modeling.perform_topic_modeling(
        user_opinions, len(set(cluster_labels.values()))
    )

    # Visualizzazione dei temi
    lda_visualizer = LDAViz()
    lda_visualizer.visualize(lda_model, corpus, dictionary, output_dir=output_dir)

    logging.info("Processo principale completato.")


if __name__ == "__main__":
    main()


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\pasqu\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\pasqu\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


'aston martin pics please follow simolude render design intelligenzaartificiale aiart aiphotography aicar aiartwork aiartcommunity aicars aiartist aicommunity carspotted automotive aidesign'